In [90]:
# psh270, jxs535, def456, hkp680

# Task 2
import numpy as np
import pandas as pd
import re
from cleantext.sklearn import CleanTransformer # likely required to ´pip install clean-text´
data = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv")

In [91]:
initial_cleaner = CleanTransformer(fix_unicode=True,               # fix various unicode errors
                                    to_ascii=True,                  # transliterate to closest ASCII representation
                                    lower=True,                     # lowercase text
                                    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
                                    no_urls=True,                  # replace all URLs with a special token
                                    no_emails=True,                # replace all email addresses with a special token
                                    no_phone_numbers=True,         # replace all phone numbers with a special token
                                    no_numbers=False,               # replace all numbers with a special token
                                    no_digits=False,                # replace all digits with a special token
                                    no_currency_symbols=True,      # replace all currency symbols with a special token
                                    no_punct=True,                 # remove punctuations
                                    replace_with_punct=" ",          # instead of removing punctuations you may replace them
                                    replace_with_url="<url>",
                                    replace_with_email="<email>",
                                    replace_with_phone_number="<phone>",
                                    replace_with_currency_symbol="<cur>",
                                    lang="en"                       # set to 'de' for German special handling
                                    )

general_cleaner = CleanTransformer(fix_unicode=False,               # fix various unicode errors
                                    to_ascii=False,                  # transliterate to closest ASCII representation
                                    lower=False,                     # lowercase text
                                    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
                                    no_urls=False,                  # replace all URLs with a special token
                                    no_emails=False,                # replace all email addresses with a special token
                                    no_phone_numbers=False,         # replace all phone numbers with a special token
                                    no_numbers=True,               # replace all numbers with a special token
                                    no_digits=False,                # replace all digits with a special token
                                    no_currency_symbols=False,      # replace all currency symbols with a special token
                                    no_punct=False,                 # remove punctuations
                                    replace_with_number="<number>",
                                    lang="en"                       # set to 'de' for German special handling
                                    )



In [92]:
three_numb_date = r'(<number> <number> <number>)' #YYYY/MM/DD or DD/MM/YYYY or MM/DD/YYYY
literal_months_date= r'(jan|feb|mar|apr|may|jun|jul|aug|sep|nov|dec)\S* ((<number> ){1,2}|([0-9]{1,2}(st|nd|rd|th)))' #Eg. jun 2nd 2020, january 23. 2021
literal_months_reverse_date = r'((<number> {1,2})|[0-9]{1,2}(st|nd|rd|th)) *(jan|feb|mar|apr|may|jun|jul|aug|sep|nov|dec)\S*' #Eg. 10th february, 4th july
all_dates = (three_numb_date) +'|' + (literal_months_date) +'|'+ (literal_months_reverse_date)
multiple_chars = r'(.)\1{3,}'
#symbols = r'([^a-z]| |^\<|^\>)' #[65-90] [97-122]

In [93]:
#string_test='In gold, the open interest SURPRISINGLY ROSE BY A CONSIDERABLE 9126 CONTRACTS UP TO582,421 WITH THE GOOD SIZED RISE IN PRICE OF GOLD WITH YESTERDAY’S TRADING ($5.55). IN ANOTHER HUGE DEVELOPMENT, WE RECEIVED THE TOTAL NUMBER OF GOLD EFP’S ISSUED FOR WEDNESDAY AND IT TOTALED A HUMONGOUS SIZED 12,223 CONTRACTS OF WHICH FEBRUARY SAW 11,023 CONTRACTS ISSUED AND APRIL SAW THE ISSUANCE OF 1200 CONTRACTS.'
#date_test  = '12/18/10 12/18/2020 12-18-10 12-18-2020 12/18/10 12/18/2020 12.18.10 12.18.2020 noise 12182010 december 18, 2010 janu 10th march 1st 3st january Dekjkj 10th  noise 10/20  noise noise 2020 10th january 2021'

def clean_column(data, col_name):
    for i, entry in enumerate(data[col_name]):    
        #We first convert to lower case and replace punctuation with space such that dates can
        #more easily be processed (eg. 10.12.2020 -> 10 12 2020 -> <NUMBER> <NUMBER> <NUMBER> instead of <NUMBER><NUMBER><DIGIT> or something)
        cleaned = initial_cleaner.transform([entry])[0]
        cleaned = general_cleaner.transform([cleaned])[0]
        cleaned = re.sub(all_dates, '<date> ', cleaned)
        #cleaned = re.sub(symbols,'',cleaned)
        #cleaned = re.sub(multiple_chars, '', cleaned)
        data.at[i, col_name] = cleaned

def clean_data(data):
    #Dropping unneeded columns
    cols_to_delete = ["Unnamed: 0","id","scraped_at","inserted_at","updated_at"]
    for column in data.columns:
        if data[column].isnull().values.all():
            cols_to_delete.append(column)

    data.drop(cols_to_delete, 1, inplace=True)
    
    clean_column(data, "content")
    clean_column(data, "title")
    return data


In [49]:
print(data["content"][9])

Subscribe to Canada Free Press for FREE

Neither Rev. Rodriguez nor Pope Francis will be there, but President Trump will be, addressing the crowd by satellite

It’s Not Really President Trump Who Needs His Mouth Washed Out With Soap



Throwing the spotlight on so-called Men of the Cloth: It’s not President Donald Trump who needs to have his mouth washed out with soap for having allegedly made use of the vulgar ‘s’ word at a White House meeting on immigration. There’s just no getting around it that some words are more painful than cuss words, particularly when they heap praise on abortion advocates. “Members of President Trump’s evangelical advisory council decided to jump ship for the day and publicly sided with Nancy Pelosi on DACA. (RedState, Jan. 18, 2018)

“You have always taken the lead. Your commitment to the immigrant community to the ‘Dreamers’ is second to none. So America is in a better place because of your prophetic leadership on this matter,” the Rev.Samuel Rodriguez, who

Since we are working on a subset of the full dataset, there is no need to include the old index/ID.
Furthermore, since pandas adds an index column, we do not have a need for the already existing (possibly error prone) local index (Unnamed: 0).

Metadata regarding scraping, time of updates etc. do not serve a significant impact on the processing we wish to perform.

In [87]:
data = clean_data(data[:10])
data

C:\Users\Computer\AppData\Local\Temp\ipykernel_26636\574242364.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop(cols_to_delete, 1, inplace=True)
C:\Users\Computer\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,domain,type,url,content,title,authors,meta_keywords,meta_description
0,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,sometimes the power of christmas will make you...,church congregation brings gift to waitresses ...,Ruth Harris,[''],NaN
1,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,awakening of <number> strands of dna reconnect...,awakening of <number> strands of dna reconnect...,Zurich Times,[''],NaN
2,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,never hike alone a friday the 13th fan film us...,never hike alone a friday the 13th fan film fu...,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...
3,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,when a rare shark was caught scientists were l...,elusive alien of the sea caught by scientist f...,Alexander Smith,[''],NaN
4,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,donald trump has the unnerving ability to abil...,trump s genius poll is complete the results ha...,Gloria Christie,[''],NaN
5,blackagendareport.com,unreliable,https://blackagendareport.com/articlelist/Medi...,republicans and democrats alike are willing to...,black agenda report,"Margaret Kimberley, Bar Editor, Senior Columni...",[''],NaN
6,awarenessact.com,conspiracy,http://awarenessact.com/tag/waking-up-in-the-m...,could you imagine waking up in the morgue i fo...,waking up in the morgue awareness act,Gerald Sinclair,[''],NaN
7,beforeitsnews.com,fake,http://beforeitsnews.com/home/featuredlist/v2_...,citizen journalist by n morgan q has released ...,citizen journalist,NaN,[''],NaN
8,beforeitsnews.com,fake,http://beforeitsnews.com/economy/2018/01/usa-d...,usa dollar tanks on mnuchin statement that he ...,usa dollar tanks on mnuchin statement that he ...,NaN,[''],NaN
9,canadafreepress.com,conspiracy,http://canadafreepress.com/article/its-not-rea...,subscribe to canada free press for free neithe...,it s not really president trump who needs his ...,"Judi Mcleod, Because Without America, There Is...",[''],NaN


In [89]:
print(data["content"][9])

subscribe to canada free press for free neither rev rodriguez nor pope francis will be there but president trump will be addressing the crowd by satellite it s not really president trump who needs his mouth washed out with soap throwing the spotlight on so called men of the cloth it s not president donald trump who needs to have his mouth washed out with soap for having allegedly made use of the vulgar s word at a white house meeting on immigration there s just no getting around it that some words are more painful than cuss words particularly when they heap praise on abortion advocates members of president trump s evangelical advisory council decided to jump ship for the day and publicly sided with nancy pelosi on daca redstate <date> you have always taken the lead your commitment to the immigrant community to the dreamers is second to none so america is in a better place because of your prophetic leadership on this matter the rev samuel rodriguez who spoke at trump s inauguration last

In [88]:
print(data["title"][9])


it s not really president trump who needs his mouth washed out with soap


In [82]:
data["title"]

0    hurch congregation brings gift to waitresses w...
1    wakening of <number> strands of dna reconnecti...
2    ever hike alone a friday the 13th fan film ful...
3    lusive alien of the sea caught by scientist fo...
4    rump s genius poll is complete the results hav...
5                                   lack agenda report
6                 aking up in the morgue awareness act
7                                    itizen journalist
8    sa dollar tanks on mnuchin statement that he w...
9    t s not really president trump who needs his m...
Name: title, dtype: object

In [4]:
# Task 3
# things to explore
# 1: clickbait type vs titles
# 2: word frequency n-gram vs article type
# 3: authors changing type

In [5]:
# Task 4
group_nr = 14
group_substring_raw = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]
group_substring = ""

for letter in np.sort(list(group_substring_raw)):
    group_substring += letter
    
group_substring

'AOPRSTUVWZ'

In [6]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://en.wikinews.org/wiki/Category:Politics_and_conflicts')
contents = response.text

soup = BeautifulSoup(contents, 'html.parser')

In [7]:
cont = soup.find_all('a', class_="external text")
links = []
for line in cont:
    if np.isin((line.attrs["href"][-1]), list(group_substring)):
        links.append(line.attrs["href"])

links = [links[0]] # Go through each href and scrape untill next h3 letter is hit. TODO
for link in links:
    response = requests.get(link)
    contents = response.text

    soup = BeautifulSoup(contents, 'html.parser')

In [22]:
cont = soup.find_all("div", id="mw-pages")